# Medical Diagnosis Finetuning: Projekt-Übersicht

## Projekt

> **"Können kleine, spezialisierte Modelle (3B Parameter) große generische Modelle (7-8B Parameter) bei der ICD-10 Klassifikation übertreffen?"**

---

### Inhaltsverzeichnis dieser Notebook-Serie

| Notebook | Titel | Beschreibung |
|----------|-------|--------------|
| **00** | Projekt-Übersicht | Forschungsfrage, Architektur, experimentelles Design |
| **01** | Daten laden & explorieren | MedSynth Dataset, Statistiken, Qualitätsprüfung |
| **02** | Datenverarbeitung & Tokenisierung | Chat-Templates, Tokenisierung, Train/Val/Test Split |
| **03** | LLM Evaluation (Zero-Shot) | Große Modelle ohne Training bewerten |
| **04** | SLM Training mit LoRA | Kleine Modelle finetunen |
| **05** | SLM Evaluation (Finetuned) | Trainierte kleine Modelle bewerten |
| **06** | Ergebnisanalyse & Vergleich | Größe vs. Spezialisierung auswerten |

---

### 1. Forschungsansatz

#### 1.1 Das Problem

Im Themenfeld der künstlichen Intelligenz stehen Experten foft vor einem fundamentalen Trade-Off:

| Ansatz | Vorteile | Nachteile |
|--------|----------|-----------|
| **Große Modelle (7-8B Parameter)** | Mehr Wissen, bessere allgemeine Performance | Langsam, teuer, hoher Ressourcenbedarf |
| **Kleine Modelle (3B Parameter)** | Schnell, günstig, lokal ausführbar | Weniger leistungsfähig ohne Training |

#### 1.2 Unsere Hypothese

> **Kann Spezialisierung durch Finetuning den Größennachteil kleinerer Modelle bei domänenspezifischen Aufgaben kompensieren?**

Anders ausgedrückt: Kann ein 3B-Modell, das auf medizinische Daten trainiert wurde, ein 8B-Modell ohne medizinisches Training übertreffen?

| Modelltyp | Größe | Training | Hypothese |
|-----------|-------|----------|-----------|
| **LLM** (Large Language Model) | 7-8B Parameter | Zero-Shot (kein Training) | Größe = Wissen |
| **SLM** (Small Language Model) | 3B Parameter | LoRA Finetuning | Spezialisierung = Effizienz |

#### 1.3 Warum ist das relevant?

1. **Kosten**: Kleinere Modelle = günstigere Inference (~3x billiger)
2. **Latenz**: Weniger Parameter = schnellere Antworten (~2.5x schneller)
3. **Datenschutz**: Kleinere Modelle können lokal laufen (On-Premise)
4. **Nachhaltigkeit**: Weniger Compute = weniger CO2-Emissionen

---

## 3. Experimentelles Design

### 3.2 Die zu vergleichenden Modelle

#### Large Language Models - Große, untrainierte Modelle

Diese Modelle werden **nicht** finetuned, sondern nur zero-shot evaluiert.
Sie repräsentieren den Ansatz "Größe ohne Spezialisierung".

| Modell | Größe | Beschreibung |
|--------|-------|--------------|
| Meta-Llama-3.1-8B-Instruct | 8B | Großes Referenzmodell von Meta |
| Mistral-7B-Instruct-v0.3 | 7B | Mittleres Referenzmodell von Mistral |

#### Small Language Models - Kleine, finetuned Modelle

| Modell | Größe | Beschreibung | Training |
|--------|-------|--------------|----------|
| Llama-3.2-3B-Instruct | 3B | Kompaktes Llama-Modell | LoRA finetuned |
| Qwen2.5-3B-Instruct | 3B | Kompaktes Qwen-Modell | LoRA finetuned |

### 3.3 Was genau testen wir?

```
LLMs (Größenvorteil)              SLMs (Spezialisierungsvorteil)
├─ Llama 8B (untrainiert)        ├─ Llama 3B (LoRA finetuned)
└─ Mistral 7B (untrainiert)      └─ Qwen 3B (LoRA finetuned)
        ↓                                 ↓
   Zero-Shot                       Domain-Adapted
   Inference                       Inference
        ↓                                 ↓
   ┌─────────────────────────────────────────┐
   │     ICD-10 Code Klassifikation          │
   │        (Medizinische Diagnose)          │
   └─────────────────────────────────────────┘
        ↓                                 ↓
   Performance                       Performance
   Vergleich                         Vergleich
```

## 4. Theoretische Grundlagen

### 4.1 Was ist Finetuning?

**Finetuning** ist das Anpassen eines vortrainierten Modells auf eine spezifische Aufgabe.

```
Vortrainiertes Modell     Finetuning        Spezialisiertes Modell
(generelles Wissen)    →  (+ Domain-Daten) →  (+ Domänen-Wissen)
```

**Warum Finetuning statt Training von Grund auf?**

- Vortrainierte Modelle haben bereits Sprachverständnis gelernt
- Finetuning braucht viel weniger Daten (1.000e vs. Milliarden)
- Schneller und günstiger

### 4.2 Was ist LoRA (Low-Rank Adaptation)?

**LoRA** ist eine **parameter-effiziente** Finetuning-Methode.

**Das Problem:** Normale Finetuning-Methoden ändern alle Parameter (Milliarden!).

**Die Lösung:** LoRA trainiert nur kleine "Adapter"-Matrizen:

```
Original-Matrix W:    [1000 x 1000] = 1.000.000 Parameter
LoRA-Matrizen A, B:   [1000 x 16] + [16 x 1000] = 32.000 Parameter
                                                   = 3.2% der Original-Größe!
```

**Die LoRA-Formel:**

$$W' = W + \Delta W = W + A \times B$$

Wobei:
- $W$ = Originale Gewichte (eingefroren, nicht trainiert)
- $A$ = Down-Projection Matrix (Input → niedrig-dimensionaler Raum)
- $B$ = Up-Projection Matrix (niedrig-dimensionaler Raum → Output)
- $r$ = Rank (typisch 8-64, kontrolliert Kapazität)

**Vorteile von LoRA:**
- Weniger als 1% der Parameter werden trainiert
- Deutlich geringerer Speicherbedarf
- Schnelleres Training
- Originales Modell bleibt unverändert (einfach rückgängig zu machen)

### 4.3 Was ist ICD-10?

**ICD-10** (International Classification of Diseases, 10. Revision) ist das weltweite Standard-System zur Klassifikation von Krankheiten und Diagnosen.

**Aufbau eines ICD-10 Codes:**

```
J06.9
│││ │
│││ └── Weitere Spezifikation (.9 = nicht näher bezeichnet)
│││
││└──── Hauptgruppe innerhalb Kapitel (06 = Akute Infektionen obere Atemwege)
││
│└───── Kapitel-Buchstabe (J = Atmungssystem)
│
└────── Hierarchie-Ebene
```

**Beispiele:**
- `J06.9` = Akute Infektion der oberen Atemwege, nicht näher bezeichnet
- `I10` = Essentielle Hypertonie (Bluthochdruck)
- `G43.9` = Migräne, nicht näher bezeichnet
- `E11.9` = Diabetes mellitus Typ 2

## 5. Technologien & Dependencies

| Framework | Zweck | Warum? |
|-----------|-------|--------|
| **PyTorch** | Deep Learning | Industry-Standard, flexible |
| **HuggingFace Transformers** | Modell-Loading | Einfacher Zugang zu vortrainierten Modellen |
| **PEFT/LoRA** | Parameter-effizientes Training | Ermöglicht Training auf Consumer-GPUs |
| **BitsAndBytes** | Quantisierung | Reduziert Speicherbedarf um 75% |
| **Datasets** | Datenverarbeitung | Effizientes Memory-Management |
| **Pydantic** | Konfiguration | Type-safe, validierte Configs |

## 6. Zentrale Konfiguration

Die folgende Zelle enthält **alle Konfigurationsparameter** für das gesamte Projekt. Diese Werte werden in allen nachfolgenden Notebooks verwendet.

In [1]:
# ============================================================
# ZENTRALE KONFIGURATION
# ============================================================
# Diese Konfiguration wird in allen Notebooks verwendet.
# ============================================================

from dataclasses import dataclass, field
from typing import List, Optional
from pathlib import Path
import torch
import os

# ============================================================
# PFAD-KONFIGURATION
# ============================================================
@dataclass
class PathConfig:
    """Alle Pfade für das Projekt."""
    # Basis-Verzeichnis (relativ zum Notebooks-Ordner)
    project_root: Path = field(default_factory=lambda: Path.cwd().parent)
    
    # Daten-Verzeichnisse
    data_dir: Path = field(default_factory=lambda: Path.cwd().parent / "data")
    cache_dir: Path = field(default_factory=lambda: Path.cwd().parent / "data" / "cache")
    
    # Modell-Verzeichnisse
    models_dir: Path = field(default_factory=lambda: Path.cwd().parent / "models")
    finetuned_models_dir: Path = field(default_factory=lambda: Path.cwd().parent / "models" / "finetuned")
    
    # Output-Verzeichnisse
    outputs_dir: Path = field(default_factory=lambda: Path.cwd().parent / "outputs")
    logs_dir: Path = field(default_factory=lambda: Path.cwd().parent / "outputs" / "logs")
    plots_dir: Path = field(default_factory=lambda: Path.cwd().parent / "outputs" / "plots")
    reports_dir: Path = field(default_factory=lambda: Path.cwd().parent / "outputs" / "reports")
    predictions_cache_dir: Path = field(default_factory=lambda: Path.cwd().parent / "outputs" / "cache" / "predictions")
    
    def create_directories(self):
        """Erstellt alle benötigten Verzeichnisse."""
        for attr_name in dir(self):
            attr = getattr(self, attr_name)
            if isinstance(attr, Path) and not attr_name.startswith('_'):
                attr.mkdir(parents=True, exist_ok=True)

# ============================================================
# DATEN-KONFIGURATION
# ============================================================
@dataclass
class DataConfig:
    """Konfiguration für Datenverarbeitung."""
    # Dataset
    dataset_name: str = "Ahmad0067/MedSynth"
    dataset_split_seed: int = 42
    
    # Train/Val/Test Split
    train_ratio: float = 0.70
    val_ratio: float = 0.15
    test_ratio: float = 0.15
    
    # Tokenisierung
    max_sequence_length: int = 512
    truncation: bool = True
    padding: str = "max_length"
    
    # Batch-Verarbeitung
    batch_size: int = 128
    num_workers: int = 12

# ============================================================
# MODELL-KONFIGURATION
# ============================================================
@dataclass
class ModelConfig:
    """Konfiguration für Modelle."""
    # LLMs (große Modelle für Zero-Shot Baseline)
    llm_models: List[dict] = field(default_factory=lambda: [
        {
            "name": "meta-llama/Meta-Llama-3.1-8B-Instruct",
            "size": "8B",
            "description": "8B - Large reference model",
            "load_in_4bit": True,
        },
        {
            "name": "mistralai/Mistral-7B-Instruct-v0.3",
            "size": "7B",
            "description": "7B - Medium reference model",
            "load_in_4bit": True,
        },
    ])
    
    # SLMs (kleine Modelle für Finetuning)
    slm_models: List[dict] = field(default_factory=lambda: [
        {
            "name": "meta-llama/Llama-3.2-3B-Instruct",
            "size": "3B",
            "description": "3B - Compact Llama for finetuning",
            "load_in_4bit": False,
        },
        {
            "name": "Qwen/Qwen2.5-3B-Instruct",
            "size": "3B",
            "description": "3B - Compact Qwen for finetuning",
            "load_in_4bit": False,
        },
    ])
    
    # Generation-Parameter
    max_new_tokens: int = 256
    temperature: float = 0.7
    top_p: float = 0.9
    top_k: int = 50
    repetition_penalty: float = 1.1

# ============================================================
# TRAINING-KONFIGURATION
# ============================================================
@dataclass
class TrainingConfig:
    """Konfiguration für LoRA Finetuning."""
    # Training Hyperparameter
    num_epochs: int = 3
    learning_rate: float = 2e-4
    warmup_steps: int = 100
    weight_decay: float = 0.01
    
    # Optimizer
    optimizer: str = "adamw_torch_fused"
    
    # Batch Sizes
    per_device_train_batch_size: int = 32
    per_device_eval_batch_size: int = 64
    gradient_accumulation_steps: int = 1
    
    # Precision
    fp16: bool = False
    bf16: bool = True
    
    # Gradient
    max_grad_norm: float = 1.0
    gradient_checkpointing: bool = False
    
    # Logging & Evaluation
    logging_steps: int = 5
    eval_steps: int = 50
    save_steps: int = 200
    save_total_limit: int = 3
    
    # Early Stopping
    early_stopping_patience: int = 3
    early_stopping_threshold: float = 0.001
    
    # LoRA Konfiguration
    use_lora: bool = True
    lora_r: int = 64
    lora_alpha: int = 128
    lora_dropout: float = 0.1
    lora_target_modules: List[str] = field(default_factory=lambda: [
        "q_proj", "v_proj", "k_proj", "o_proj",
        "gate_proj", "up_proj", "down_proj"
    ])
    
    # Reproducibility
    seed: int = 42

# ============================================================
# EVALUATION-KONFIGURATION
# ============================================================
@dataclass
class EvaluationConfig:
    """Konfiguration für Evaluation."""
    eval_batch_size: int = 48
    max_eval_samples: Optional[int] = None
    use_prediction_cache: bool = True
    generate_plots: bool = True

# ============================================================
# HAUPT-KONFIGURATION
# ============================================================
@dataclass
class Config:
    """Haupt-Konfiguration die alle Sub-Configs vereint."""
    paths: PathConfig = field(default_factory=PathConfig)
    data: DataConfig = field(default_factory=DataConfig)
    model: ModelConfig = field(default_factory=ModelConfig)
    training: TrainingConfig = field(default_factory=TrainingConfig)
    evaluation: EvaluationConfig = field(default_factory=EvaluationConfig)
    
    def setup(self):
        """Initialisiert alle Verzeichnisse und Seeds."""
        self.paths.create_directories()
        self._set_seeds()
        self._enable_tf32()
    
    def _set_seeds(self):
        """Setzt alle Random Seeds."""
        import random
        import numpy as np
        
        seed = self.training.seed
        random.seed(seed)
        np.random.seed(seed)
        torch.manual_seed(seed)
        if torch.cuda.is_available():
            torch.cuda.manual_seed_all(seed)
    
    def _enable_tf32(self):
        """Aktiviert TF32 für NVIDIA GPUs."""
        if torch.cuda.is_available():
            torch.backends.cuda.matmul.allow_tf32 = True
            torch.backends.cudnn.allow_tf32 = True

# ============================================================
# KONFIGURATION ERSTELLEN
# ============================================================
def get_config() -> Config:
    """Factory-Funktion für Konfiguration."""
    return Config()

# Initialisierung
config = get_config()
config.setup()

print("Konfiguration geladen!")
print(f"   Projekt-Root: {config.paths.project_root}")
print(f"   Dataset: {config.data.dataset_name}")
print(f"   CUDA verfügbar: {torch.cuda.is_available()}")
if torch.cuda.is_available():
    print(f"   GPU: {torch.cuda.get_device_name(0)}")

Konfiguration geladen!
   Projekt-Root: /home/bmw/src/simon/finetuning
   Dataset: Ahmad0067/MedSynth
   CUDA verfügbar: True
   GPU: NVIDIA GeForce RTX 5090


### Utility-Funktionen

Die folgenden Funktionen werden in allen Notebooks verwendet:

In [2]:
# ============================================================
# UTILITY-FUNKTIONEN
# ============================================================
# Diese Funktionen werden in allen Notebooks verwendet.
# ============================================================

import gc
import json
import hashlib
import logging
from datetime import datetime, timedelta
from typing import Dict, Any, Optional

def get_device() -> str:
    """Bestimmt das beste verfügbare Device."""
    if torch.cuda.is_available():
        return "cuda"
    elif hasattr(torch.backends, 'mps') and torch.backends.mps.is_available():
        return "mps"
    return "cpu"

def aggressive_memory_cleanup():
    """Führt aggressive Memory-Cleanup durch."""
    gc.collect()
    if torch.cuda.is_available():
        torch.cuda.empty_cache()
        torch.cuda.synchronize()
        torch.cuda.ipc_collect()

def log_gpu_memory(prefix: str = ""):
    """Loggt aktuellen GPU Memory Status."""
    if not torch.cuda.is_available():
        return
    
    allocated = torch.cuda.memory_allocated() / 1024**3
    reserved = torch.cuda.memory_reserved() / 1024**3
    
    msg = f"GPU Memory: {allocated:.2f}GB allocated, {reserved:.2f}GB reserved"
    if prefix:
        msg = f"{prefix} - {msg}"
    print(msg)

def save_json(data: dict, path: Path):
    """Speichert Dict als JSON."""
    with open(path, "w") as f:
        json.dump(data, f, indent=2, default=str)

def load_json(path: Path) -> dict:
    """Lädt JSON als Dict."""
    with open(path, "r") as f:
        return json.load(f)

def generate_cache_key(model_name: str, dataset_size: int, 
                       generation_config: Optional[Dict[str, Any]] = None) -> str:
    """Generiert eindeutigen Cache-Key für Predictions."""
    cache_parts = [model_name.replace("/", "_"), str(dataset_size)]
    
    if generation_config:
        config_str = json.dumps(generation_config, sort_keys=True)
        cache_parts.append(config_str)
    
    cache_string = "|".join(cache_parts)
    cache_hash = hashlib.md5(cache_string.encode()).hexdigest()[:12]
    
    model_short = model_name.split("/")[-1].replace(".", "_")
    return f"{model_short}_{cache_hash}"

print("Utility-Funktionen geladen!")
print(f"   Device: {get_device()}")

Utility-Funktionen geladen!
   Device: cuda


## Pipeline-Übersicht

```
┌─────────────────────────────────────────────────────────────────────┐
│                        MEDICAL DIAGNOSIS PIPELINE                    │
├─────────────────────────────────────────────────────────────────────┤
│                                                                      │
│  ┌─────────────┐   ┌─────────────┐   ┌─────────────┐                │
│  │   MedSynth  │──▶│   Tokenize  │──▶│ Train/Val/  │                │
│  │   Dataset   │   │  + Format   │   │ Test Split  │                │
│  └─────────────┘   └─────────────┘   └──────┬──────┘                │
│                                              │                       │
│         ┌────────────────────────────────────┼──────────┐           │
│         ▼                                    ▼          ▼           │
│  ┌─────────────┐                      ┌─────────────┐               │
│  │  LLM (8B)   │                      │  SLM (3B)   │               │
│  │  Zero-Shot  │                      │   LoRA FT   │               │
│  └──────┬──────┘                      └──────┬──────┘               │
│         │                                    │                       │
│         ▼                                    ▼                       │
│  ┌─────────────┐                      ┌─────────────┐               │
│  │  Evaluate   │                      │  Evaluate   │               │
│  │  Baseline   │                      │  Finetuned  │               │
│  └──────┬──────┘                      └──────┬──────┘               │
│         │                                    │                       │
│         └────────────────┬───────────────────┘                      │
│                          ▼                                          │
│                   ┌─────────────┐                                   │
│                   │   Compare   │                                   │
│                   │  & Report   │                                   │
│                   └─────────────┘                                   │
│                                                                      │
└─────────────────────────────────────────────────────────────────────┘
```

---